 # **Predict Future Sales**

## **Task**
To predict total sales for every product and store in the next month.

## **File descriptions**
* sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
* test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
* sample_submission.csv - a sample submission file in the correct format.
* items.csv - supplemental information about the items/products.
* item_categories.csv  - supplemental information about the items categories.
* shops.csv- supplemental information about the shops.

In [1]:
import numpy as np 
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/competitive-data-science-predict-future-sales/items.csv
/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv
/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv
/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv
/kaggle/input/competitive-data-science-predict-future-sales/shops.csv
/kaggle/input/competitive-data-science-predict-future-sales/test.csv


In [2]:
item=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
submission=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
item_cat=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
train=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
shop=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
test=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')

In [3]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [5]:
train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [6]:
train.nunique()

date               1034
date_block_num       34
shop_id              60
item_id           21807
item_price        19993
item_cnt_day        198
dtype: int64

In [7]:
test.nunique()

ID         214200
shop_id        42
item_id      5100
dtype: int64

In [8]:
train['date']=pd.to_datetime(train['date'],dayfirst=True)
train['date']=train['date'].apply(lambda x : x.strftime("%Y-%m"))
train['date']

0          2013-01
1          2013-01
2          2013-01
3          2013-01
4          2013-01
            ...   
2935844    2015-10
2935845    2015-10
2935846    2015-10
2935847    2015-10
2935848    2015-10
Name: date, Length: 2935849, dtype: object

In [9]:
train.drop(['date_block_num','item_price'],axis=1,inplace=True)

In [10]:
train=train.groupby(['date','shop_id','item_id']).sum()
train=train.pivot_table(index=['shop_id','item_id'],columns='date',values='item_cnt_day',fill_value=0)
train.reset_index(inplace=True)

In [11]:
train.head()

date,shop_id,item_id,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,...,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10
0,0,30,0,31,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,31,0,11,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,32,6,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,33,3,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,35,1,14,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
train=pd.merge(train,shop,how='left',on='shop_id')
train=pd.merge(train,item,how='left',on='item_id')
train=pd.merge(train,item_cat,how='left',on='item_category_id')
train.head()

,shop_id,item_id,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,...,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,shop_name,item_name,item_category_id,item_category_name
0,0,30,0,31,0,0,0,0,0,0,...,0,0,0,0,0,0,"!Якутск Орджоникидзе, 56 фран",007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,Кино - DVD
1,0,31,0,11,0,0,0,0,0,0,...,0,0,0,0,0,0,"!Якутск Орджоникидзе, 56 фран",007: КООРДИНАТЫ «СКАЙФОЛЛ» (BD),37,Кино - Blu-Ray
2,0,32,6,10,0,0,0,0,0,0,...,0,0,0,0,0,0,"!Якутск Орджоникидзе, 56 фран",1+1,40,Кино - DVD
3,0,33,3,3,0,0,0,0,0,0,...,0,0,0,0,0,0,"!Якутск Орджоникидзе, 56 фран",1+1 (BD),37,Кино - Blu-Ray
4,0,35,1,14,0,0,0,0,0,0,...,0,0,0,0,0,0,"!Якутск Орджоникидзе, 56 фран",10 ЛЕТ СПУСТЯ,40,Кино - DVD


In [13]:
test=pd.merge(test,train,how='left',on=['shop_id','item_id'])
test.drop(['ID','2013-01'],axis=1,inplace=True)
test=test.fillna(0)
test.head()

,shop_id,item_id,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,...,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,shop_name,item_name,item_category_id,item_category_name
0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,3.0,1.0,0.0,"Вологда ТРЦ ""Мармелад""","NHL 15 [PS3, русские субтитры]",19.0,Игры - PS3
1,5,5320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0
2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,2.0,0.0,1.0,3.0,1.0,"Вологда ТРЦ ""Мармелад""","Need for Speed Rivals (Essentials) [PS3, русск...",19.0,Игры - PS3
3,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,"Вологда ТРЦ ""Мармелад""","Need for Speed Rivals (Classics) [Xbox 360, ру...",23.0,Игры - XBOX 360
4,5,5268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0


In [14]:
train.rename(columns={'shop_name_x':'shop_name'},inplace=True)

In [15]:
test.drop('shop_name_y',axis=1,inplace=True)
test.rename(columns={'shop_name_x':'shop_name'},inplace=True)
test.head()

KeyError: "['shop_name_y'] not found in axis"

In [ ]:
from sklearn.preprocessing import LabelEncoder
for i in ['shop_name','item_name','item_category_name']:
    label=LabelEncoder()
    label.fit(list(train[i].unique())+list(test[i].unique()))
    train[i]=label.transform(train[i].astype(str))
    test[i]=label.transform(test[i].astype(str))
    print(i)

In [ ]:
X_train=train.drop(['2015-10'],axis=1)
y_train=train['2015-10'].values

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr=RandomForestRegressor(n_estimators=100,n_jobs=1,random_state=0,verbose=1)
rfr.fit(X_train,y_train)
predictions=rfr.predict(test)
predictions

In [ ]:
submission=pd.DataFrame({'ID': np.arange(0,len(predictions)),'item_cnt_month':predictions})
submission.head()

In [ ]:
submission.to_csv('submission.csv',index=False)